In [75]:
from __future__ import print_function, absolute_import, division
from keras import backend as K
from keras.engine.base_layer import Layer
from keras.engine.topology import InputSpec
import tensorflow as tf
from keras.utils import conv_utils
from keras.layers import Input, MaxPooling1D ,Activation,Dense, activations, initializers, Flatten, regularizers, constraints, Lambda,Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.utils import plot_model
import numpy as np
from scipy.fftpack import dct
from keras.backend.common import normalize_data_format
from keras.layers.merge import Multiply
from keras.utils import to_categorical

In [50]:
debug = False
class Attention(Layer):
    '''Custom Layer for ResNet used for BatchNormalization.

    Linear learnable weight vector , does dot multiplication on a vector
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).'''

    def __init__(self, weights=None, axis=-1,init='he_normal',**kwargs):
        if(debug):
            print("Init")
        self.axis = axis
        self.init = initializers.get(init)
        self.kernel = weights
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        if len(input_shape)>2:
            raise ValueError("Input to attention layer hasn't been flattened")
        self.input_dim = input_shape[-1]
        if(debug):print("DIM ", input_shape, len(input_shape))
            
        self.kernel = self.add_weight(shape=(self.input_dim,),
                                      initializer=self.init,
                                      name='kernel',
                                      constraint=constraints.NonNeg())
                                      #constraint=constraints.UnitNorm(axis=self.axis))
        self.input_spec = InputSpec(min_ndim=2, axes={-1: self.input_dim})
        if(debug):print("Building : kernela shape",self.kernel.shape)
            
        self.built = True

    def call(self, inputs):
        if(debug):
            print("input ", inputs)
            print("kernel ",self.kernel)
        output = tf.multiply(inputs,self.kernel)
        if(debug):
            print("OUt put " ,type(output))
        return output
    def compute_output_shape(self, input_shape):
        return input_shape
    def get_config(self):
        config = {"kernel": self.kernel, "axis": self.axis}
        base_config = super(Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [119]:
att = Attention(name = 'wow')

In [116]:
out = att(b)

In [117]:
print((tf.Session().run(out)))

FailedPreconditionError: Attempting to use uninitialized value wow_14/kernel
	 [[node wow_14/kernel/read (defined at C:\Users\Ansari\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:402)  = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](wow_14/kernel)]]
	 [[{{node wow_14/Mul/_1}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_8_wow_14/Mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'wow_14/kernel/read', defined at:
  File "C:\Users\Ansari\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Ansari\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Ansari\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance

In [115]:
a = tf.convert_to_tensor(np.array([1,2,3]).astype(np.float32))
b = tf.convert_to_tensor(np.array([1,2,3]).astype(np.float32).reshape(1,3))
c = tf.convert_to_tensor(np.array([[1,2,3],[4,5,6]]).astype(np.float32).reshape(2,3))
a,b,c

(<tf.Tensor 'Const:0' shape=(3,) dtype=float32>,
 <tf.Tensor 'Const_1:0' shape=(1, 3) dtype=float32>,
 <tf.Tensor 'Const_2:0' shape=(2, 3) dtype=float32>)

In [8]:
out = Lambda(lambda x: x[0] * K.expand_dims(x[1], axis=-1))([a, b])

In [9]:
np.array(out)

array(<tf.Tensor 'lambda_1/mul:0' shape=(1, 3, 3) dtype=float32>,
      dtype=object)

In [10]:
out = tf.multiply(c,a)

In [13]:
out = Multiply()([b,b])

In [14]:
print((tf.Session().run(out)))

[[1. 4. 9.]]


In [49]:
att.get_weights()

[array([-0.1099719,  0.4618892,  0.4047528], dtype=float32)]

In [46]:
-.09050519+.05788201-.47705907

-0.50968225

In [50]:
att.get_config()

{'name': 'attention_6',
 'trainable': True,
 'kernel': <tf.Variable 'attention_6/kernel:0' shape=(3,) dtype=float32_ref>,
 'axis': -1}

In [105]:
def Network():
    input = Input(shape=(10, 1))
    t = BatchNormalization(epsilon=1.1e-5, momentum=.99, axis=-1)(input)
    t = Activation('relu')(t)
    t = Dropout(rate=.5, seed=1)(t)
    t = MaxPooling1D(pool_size=1)(t)
    
    t = Flatten()(t)
    tt = t
    t = Attention(name='att')(t)
    t2 = Attention(name='att2')(tt)
    t = Dense(2,activation='softmax',name='out')(t)
    t2 = Dense(2,activation='softmax',name='out2')(t2)
    opt = SGD(lr=.01,decay=.001)
    model = Model(inputs=input, outputs=[t,t2])
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [106]:
model = Network()

In [107]:
model.summary()
plot_model(model, to_file='model.png', show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 10, 1)        0                                            
__________________________________________________________________________________________________
batch_normalization_19 (BatchNo (None, 10, 1)        4           input_19[0][0]                   
__________________________________________________________________________________________________
activation_19 (Activation)      (None, 10, 1)        0           batch_normalization_19[0][0]     
__________________________________________________________________________________________________
dropout_19 (Dropout)            (None, 10, 1)        0           activation_19[0][0]              
__________________________________________________________________________________________________
max_poolin

In [108]:
model.get_layer("att" ).get_config()

{'name': 'att',
 'trainable': True,
 'kernel': <tf.Variable 'att_17/kernel:0' shape=(10,) dtype=float32_ref>,
 'axis': -1}

In [125]:
x = np.random.rand(5,10,1)
y = np.random.randint(low = 0,high = 2,size = (5,1))
y = to_categorical(y,num_classes=2)

In [110]:
model.fit(x,[y,y],epochs=1,verbose=1)

Epoch 1/1
5/5 [==============================] - 1s 143ms/step - loss: 2.3124 - out_loss: 1.1709 - out2_loss: 1.1415 - out_acc: 0.6000 - out2_acc: 0.2000


In [136]:
model.get_layer("att" ).get_weights(),model.get_layer("att2" ).get_weights()

([array([ 0.1290113 ,  0.77499616,  0.48132297, -0.        , -0.        ,
         -0.        ,  0.893297  ,  1.5842648 ,  0.38220668, -0.        ],
        dtype=float32)],
 [array([-0.        ,  0.19380453,  0.70915276,  0.99227256,  0.7119882 ,
         -0.        , -0.        ,  1.4469829 , -0.        , -0.        ],
        dtype=float32)])

In [112]:
for x in (hm[0]):
    print(x)

0.1290113
0.77499616
0.48132297
-0.0
-0.0
-0.0
0.893297
1.5842648
0.38220668
-0.0


In [129]:
a = tf.convert_to_tensor(np.random.rand(5,10).astype(np.float32))

In [130]:
att(a)

<tf.Tensor 'wow_19/Mul:0' shape=(5, 10) dtype=float32>

In [137]:
att.set_weights(model.get_layer("att" ).get_weights())

In [138]:
att.get_weights()

[array([ 0.1290113 ,  0.77499616,  0.48132297, -0.        , -0.        ,
        -0.        ,  0.893297  ,  1.5842648 ,  0.38220668, -0.        ],
       dtype=float32)]

In [135]:
att.get_weights()

[array([-0.2323829 , -0.24704112, -0.5642373 , -1.3949541 ,  0.1574892 ,
        -0.84826344, -1.3944159 ,  1.6270232 , -0.6745873 , -0.13708325],
       dtype=float32)]